# Case

**Business Problem:** FLO wants to determine a roadmap for its sales and marketing activities. In order for the company to make a medium-long term plan, the potential value that existing customers will provide to the company in the future needs to be estimated.

In [1]:
pip install lifetimes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.2/584.2 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 2.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [3]:
df = pd.read_csv("/kaggle/input/flo-cltv/flo_data_20k.csv")
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0000,1.0000,139.9900,799.3800,[KADIN]
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0000,2.0000,159.9700,1853.5800,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0000,2.0000,189.9700,395.3500,"[ERKEK, KADIN]"
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0000,1.0000,39.9900,81.9800,"[AKTIFCOCUK, COCUK]"
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0000,1.0000,49.9900,159.9900,[AKTIFSPOR]


# Preparing Data 

In [4]:
def check_df(dataframe, head=5):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Duplicated Values #####################")
    print(dataframe.duplicated().sum())
    print("##################### Missing Values #####################")
    print(dataframe.isnull().sum())
    print("##################### Number of Unique Values #####################")
    print(df.nunique())
    
check_df(df)

##################### Shape #####################
(19945, 12)
##################### Types #####################
master_id                             object
order_channel                         object
last_order_channel                    object
first_order_date                      object
last_order_date                       object
last_order_date_online                object
last_order_date_offline               object
order_num_total_ever_online          float64
order_num_total_ever_offline         float64
customer_value_total_ever_offline    float64
customer_value_total_ever_online     float64
interested_in_categories_12           object
dtype: object
##################### Duplicated Values #####################
0
##################### Missing Values #####################
master_id                            0
order_channel                        0
last_order_channel                   0
first_order_date                     0
last_order_date                      0
last_order_date_

In [5]:
for i in df.columns:
    if "date" in i:
        df[i]=pd.to_datetime(df[i])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   master_id                          19945 non-null  object        
 1   order_channel                      19945 non-null  object        
 2   last_order_channel                 19945 non-null  object        
 3   first_order_date                   19945 non-null  datetime64[ns]
 4   last_order_date                    19945 non-null  datetime64[ns]
 5   last_order_date_online             19945 non-null  datetime64[ns]
 6   last_order_date_offline            19945 non-null  datetime64[ns]
 7   order_num_total_ever_online        19945 non-null  float64       
 8   order_num_total_ever_offline       19945 non-null  float64       
 9   customer_value_total_ever_offline  19945 non-null  float64       
 10  customer_value_total_ever_online  

In [6]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
first_order_date,19945,2019-03-22 16:43:55.246929152,2013-01-14 00:00:00,2019-02-16 00:00:00,2019-08-20 00:00:00,2020-01-01 00:00:00,2021-05-27 00:00:00,NaN
last_order_date,19945,2021-01-17 12:59:57.653547264,2020-05-30 00:00:00,2020-11-11 00:00:00,2021-02-10 00:00:00,2021-04-19 00:00:00,2021-05-30 00:00:00,NaN
last_order_date_online,19945,2020-08-10 01:08:52.644773376,2014-02-24 00:00:00,2020-06-18 00:00:00,2020-12-14 00:00:00,2021-04-05 00:00:00,2021-05-30 00:00:00,NaN
last_order_date_offline,19945,2020-05-17 12:07:24.021058048,2019-01-22 00:00:00,2019-11-04 00:00:00,2020-06-22 00:00:00,2020-12-25 00:00:00,2021-05-28 00:00:00,NaN
order_num_total_ever_online,19945.0000,3.1109,1.0000,1.0000,2.0000,4.0000,200.0000,4.2256
order_num_total_ever_offline,19945.0000,1.9139,1.0000,1.0000,1.0000,2.0000,109.0000,2.0629
customer_value_total_ever_offline,19945.0000,253.9226,10.0000,99.9900,179.9800,319.9700,18119.1400,301.5329
customer_value_total_ever_online,19945.0000,497.3217,12.9900,149.9800,286.4600,578.4400,45220.1300,832.6019


# Outlier Analysis

In [7]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = round(low_limit, 0)
    dataframe.loc[(dataframe[variable] > up_limit), variable] = round(up_limit, 0)

In [8]:
replace_with_thresholds(df,"order_num_total_ever_online")
replace_with_thresholds(df,"order_num_total_ever_offline")
replace_with_thresholds(df,"customer_value_total_ever_offline")
replace_with_thresholds(df,"customer_value_total_ever_online")
df.dropna(inplace=True)
df.describe().T

,count,mean,min,25%,50%,75%,max,std
first_order_date,19945,2019-03-22 16:43:55.246929152,2013-01-14 00:00:00,2019-02-16 00:00:00,2019-08-20 00:00:00,2020-01-01 00:00:00,2021-05-27 00:00:00,NaN
last_order_date,19945,2021-01-17 12:59:57.653547264,2020-05-30 00:00:00,2020-11-11 00:00:00,2021-02-10 00:00:00,2021-04-19 00:00:00,2021-05-30 00:00:00,NaN
last_order_date_online,19945,2020-08-10 01:08:52.644773376,2014-02-24 00:00:00,2020-06-18 00:00:00,2020-12-14 00:00:00,2021-04-05 00:00:00,2021-05-30 00:00:00,NaN
last_order_date_offline,19945,2020-05-17 12:07:24.021058048,2019-01-22 00:00:00,2019-11-04 00:00:00,2020-06-22 00:00:00,2020-12-25 00:00:00,2021-05-28 00:00:00,NaN
order_num_total_ever_online,19945.0000,3.0920,1.0000,1.0000,2.0000,4.0000,48.0000,3.8095
order_num_total_ever_offline,19945.0000,1.8862,1.0000,1.0000,1.0000,2.0000,16.0000,1.4347
customer_value_total_ever_offline,19945.0000,251.9213,10.0000,99.9900,179.9800,319.9700,3020.0000,251.0237
customer_value_total_ever_online,19945.0000,489.7057,12.9900,149.9800,286.4600,578.4400,7800.0000,632.6098


In [9]:
df["Omnichannel_Counts"]=df["order_num_total_ever_online"]+df["order_num_total_ever_offline"]
df["Omnichannel_Price"]=df["customer_value_total_ever_offline"]+df["customer_value_total_ever_online"]

# Creating CLTV Data Structure

In [10]:
df["last_order_date"].max()

Timestamp('2021-05-30 00:00:00')

In [11]:
today_date=dt.datetime(2021,6,1)

In [12]:
cltv_df=pd.DataFrame()

cltv_df["master_id"]=df["master_id"]
cltv_df["recency_cltv_weekly"]=(df["last_order_date"]-df["first_order_date"]).dt.days
cltv_df["T_weekly"]=(today_date-df["first_order_date"]).dt.days
cltv_df["frequency"]=df["Omnichannel_Counts"]
cltv_df["monetary_cltv_avg"]=df["Omnichannel_Price"]

cltv_df.head()

,master_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg
0,cc294636-19f0-11eb-8d74-000d3a38a36f,119,214,5.0000,939.3700
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,1469,1574,21.0000,2013.5500
2,69b69676-1a40-11ea-941b-000d3a38a36f,366,552,5.0000,585.3200
3,1854e56c-491f-11eb-806e-000d3a38a36f,11,146,2.0000,121.9700
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,582,668,2.0000,209.9800


In [13]:
cltv_df["recency_cltv_weekly"]=cltv_df["recency_cltv_weekly"]/7
cltv_df["T_weekly"]=cltv_df["T_weekly"]/7
cltv_df["monetart_cltv_avg"]=cltv_df["monetary_cltv_avg"]/cltv_df["frequency"]
cltv_df=cltv_df[cltv_df["frequency"] >1 ]
cltv_df.head()

,master_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,monetart_cltv_avg
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.5714,5.0000,939.3700,187.8740
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,224.8571,21.0000,2013.5500,95.8833
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,78.8571,5.0000,585.3200,117.0640
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,20.8571,2.0000,121.9700,60.9850
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.4286,2.0000,209.9800,104.9900


# Establishing BG/NBD, Gamma-Gamma Models

In [14]:
bgf=BetaGeoFitter(penalizer_coef=0.001)

bgf.fit(cltv_df["frequency"],
        cltv_df["recency_cltv_weekly"],
        cltv_df["T_weekly"])

/opt/conda/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


<lifetimes.BetaGeoFitter: fitted with 19945 subjects, a: 0.00, alpha: 76.17, b: 0.00, r: 3.66>

In [15]:
# Expected purchases from customers within 3 months

cltv_df["exp_sales_3_months"]=bgf.conditional_expected_number_of_purchases_up_to_time(4*3, 
                                                        cltv_df["frequency"],
                                                        cltv_df["recency_cltv_weekly"],
                                                        cltv_df["T_weekly"])
cltv_df.head()

,master_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,monetart_cltv_avg,exp_sales_3_months
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.5714,5.0000,939.3700,187.8740,0.9739
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,224.8571,21.0000,2013.5500,95.8833,0.9832
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,78.8571,5.0000,585.3200,117.0640,0.6706
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,20.8571,2.0000,121.9700,60.9850,0.7004
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.4286,2.0000,209.9800,104.9900,0.3960


In [16]:
# Expected purchases from customers within 6 months

cltv_df["exp_Sales_6_month"]=bgf.conditional_expected_number_of_purchases_up_to_time(4*6, cltv_df["frequency"], cltv_df["recency_cltv_weekly"],cltv_df["T_weekly"])

cltv_df.head()

,master_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,monetart_cltv_avg,exp_sales_3_months,exp_Sales_6_month
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.5714,5.0000,939.3700,187.8740,0.9739,1.9479
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,224.8571,21.0000,2013.5500,95.8833,0.9832,1.9663
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,78.8571,5.0000,585.3200,117.0640,0.6706,1.3412
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,20.8571,2.0000,121.9700,60.9850,0.7004,1.4008
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.4286,2.0000,209.9800,104.9900,0.3960,0.7921


In [17]:
replace_with_thresholds(cltv_df,"frequency")

ggf=GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(cltv_df["frequency"],cltv_df["monetary_cltv_avg"])
cltv_df["exp_avg_value"]=ggf.conditional_expected_average_profit(cltv_df["frequency"], cltv_df["monetary_cltv_avg"])
cltv_df.head()

,master_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,monetart_cltv_avg,exp_sales_3_months,exp_Sales_6_month,exp_avg_value
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.5714,5.0000,939.3700,187.8740,0.9739,1.9479,979.8405
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,224.8571,21.0000,2013.5500,95.8833,0.9832,1.9663,2033.3667
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,78.8571,5.0000,585.3200,117.0640,0.6706,1.3412,610.8127
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,20.8571,2.0000,121.9700,60.9850,0.7004,1.4008,137.6955
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.4286,2.0000,209.9800,104.9900,0.3960,0.7921,235.6442


In [18]:
# Calculating 6 months CLTV

cltv_df["cltv"]=ggf.customer_lifetime_value(bgf,
                                 cltv_df["frequency"],
                                 cltv_df["recency_cltv_weekly"],
                                 cltv_df["T_weekly"],
                                 cltv_df["monetary_cltv_avg"],
                                 time=6*4,  
                                 freq="W",
                                 discount_rate=0.01 
                                 )
cltv_df.head()

,master_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,monetart_cltv_avg,exp_sales_3_months,exp_Sales_6_month,exp_avg_value,cltv
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.5714,5.0000,939.3700,187.8740,0.9739,1.9479,979.8405,7340.3008
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,224.8571,21.0000,2013.5500,95.8833,0.9832,1.9663,2033.3667,15377.0437
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,78.8571,5.0000,585.3200,117.0640,0.6706,1.3412,610.8127,3150.6110
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,20.8571,2.0000,121.9700,60.9850,0.7004,1.4008,137.6955,741.8320
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.4286,2.0000,209.9800,104.9900,0.3960,0.7921,235.6442,717.8382


In [19]:
# 20 people with the highest cltv value

cltv_df.sort_values(by="cltv",ascending=False).head(20)

,master_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,monetart_cltv_avg,exp_sales_3_months,exp_Sales_6_month,exp_avg_value,cltv
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,62.7143,67.2857,52.0000,8643.6800,166.2246,4.6561,9.3123,8677.6128,310783.8368
4315,d5ef8058-a5c6-11e9-a2fc-000d3a38a36f,133.1429,147.1429,49.0000,7930.4900,161.8467,2.8299,5.6598,7963.5429,173344.6003
19538,55d54d9e-8ac7-11ea-8ec0-000d3a38a36f,52.5714,58.7143,31.0000,7084.4300,228.5300,3.0838,6.1676,7131.2264,169153.0270
14181,0e8c7b30-ac18-11e9-a2fc-000d3a38a36f,141.7143,159.7143,48.0000,8181.1900,170.4415,2.6282,5.2564,8215.9989,166093.7970
10489,7af5cd16-b100-11e9-9757-000d3a38a36f,103.1429,111.8571,43.0000,6755.8400,157.1126,2.9780,5.9561,6787.9569,155490.1401
10536,e143b6fa-d6f8-11e9-93bc-000d3a38a36f,104.5714,113.4286,40.0000,7048.0000,176.2000,2.7635,5.5270,7084.0277,150581.1899
14330,90d628b6-1928-11ea-b6a2-000d3a38a36f,98.5714,117.4286,32.0000,7879.9800,246.2494,2.2105,4.4211,7930.3820,134841.2477
4157,7eed6468-4540-11ea-acaf-000d3a38a36f,89.1429,90.0000,27.0000,7823.6300,289.7641,2.2143,4.4287,7883.0097,134266.2625
8068,d696c654-2633-11ea-8e1c-000d3a38a36f,204.8571,211.8571,49.0000,7882.4800,160.8669,2.1941,4.3882,7915.3333,133584.0296
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,6.1429,13.1429,11.0000,8338.9400,758.0855,1.9701,3.9402,8495.9990,128746.8893


# Segmentation

In [20]:
cltv_df["segment"]=pd.qcut(cltv_df["cltv"],q=4,labels=["D","C","B","A"])
cltv_df.head()

,master_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,monetart_cltv_avg,exp_sales_3_months,exp_Sales_6_month,exp_avg_value,cltv,segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.5714,5.0000,939.3700,187.8740,0.9739,1.9479,979.8405,7340.3008,A
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,224.8571,21.0000,2013.5500,95.8833,0.9832,1.9663,2033.3667,15377.0437,A
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,78.8571,5.0000,585.3200,117.0640,0.6706,1.3412,610.8127,3150.6110,B
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,20.8571,2.0000,121.9700,60.9850,0.7004,1.4008,137.6955,741.8320,D
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.4286,2.0000,209.9800,104.9900,0.3960,0.7921,235.6442,717.8382,D
